In [11]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size=(2,4))
net(X)

tensor([[-0.2319],
        [-0.3035]], grad_fn=<AddmmBackward0>)

我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数。

In [4]:
print(net[2].state_dict())
# 首先，这个全连接层包含两个参数，分别是该层的权重和偏置。 
# 两者都存储为单精度浮点数（float32）。
# 注意，参数名称允许唯一标识每个参数，
# 即使在包含数百个层的网络中也是如此。

OrderedDict([('weight', tensor([[ 0.2794, -0.3522,  0.1469, -0.3466, -0.1618, -0.2546, -0.2341,  0.3195]])), ('bias', tensor([0.1722]))])


In [14]:
print(type(net[2].bias))
print(net[2].weight.data)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
tensor([[-0.3379,  0.0343,  0.0667, -0.3219, -0.1211, -0.0334, -0.2801, -0.3085]])
tensor([0.0932])


In [15]:
net[2].weight.grad == None

True

### 一次性访问所有参数

In [17]:
print(*[(name,param) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', Parameter containing:
tensor([[-0.1063, -0.0299, -0.1067,  0.0025],
        [ 0.0431,  0.2571,  0.1439, -0.0871],
        [-0.4033,  0.2817,  0.3047, -0.4767],
        [-0.0942,  0.3529, -0.0923, -0.2050],
        [ 0.3709, -0.3645,  0.2480,  0.3477],
        [ 0.4777,  0.1357, -0.4417,  0.0689],
        [ 0.4416,  0.1100, -0.2249,  0.2679],
        [-0.0605,  0.1342,  0.2585,  0.0296]], requires_grad=True)) ('bias', Parameter containing:
tensor([ 0.3637,  0.0831, -0.2838, -0.1819, -0.3340,  0.3951,  0.0918,  0.3446],
       requires_grad=True))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [18]:
net.state_dict()['2.bias'].data

tensor([0.0932])

### 从嵌套块收集参数

In [20]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                        nn.Linear(8,4),nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}',block1())
    return net
rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.2486],
        [0.2486]], grad_fn=<AddmmBackward0>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [24]:
rgnet[0][1][0].bias.data

tensor([ 0.0278,  0.1057, -0.1544, -0.2458,  0.0792, -0.2500, -0.1904, -0.0477])

### 5.2.2. 参数初始化
#### 5.2.2.1. 内置初始化

In [25]:
# 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
# 且将偏置参数设置为0。
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0119, -0.0175,  0.0035, -0.0076]), tensor(0.))

In [27]:
# 我们还可以将所有参数初始化为给定的常数，比如初始化为1。
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

有时，深度学习框架没有提供我们需要的初始化方法。 在下面的例子中，我们使用以下的分布为任意权重参数定义初始化方法：
\begin{split}\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}\end{split}

In [28]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10) # 均匀分布
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -9.7666,  8.8690, -0.0000],
        [ 9.1113,  0.0000,  0.0000, -6.1846]], grad_fn=<SliceBackward0>)

我们还可以对某些块应用不同的初始化方法。 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42

In [30]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.3966, 0.0261, 0.4437, 0.0011])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 5.2.3. 参数绑定

In [32]:
# 有时我们希望在多个层间共享参数： 
# 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 第三个和第五个神经网络层的参数是绑定的。 它们不仅值相等，而且由相同的张量表示。
# 因此，如果我们改变其中一个参数，另一个参数也会改变。
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


### 5.2.4. 小结
- 我们有几种方法可以访问、初始化和绑定模型参数。

- 我们可以使用自定义初始化方法。